<a href="https://colab.research.google.com/github/wahid028/Sentiment-Analysis/blob/main/sentiment_analysis_using_BERT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install kaggle
!pip install -q kaggle

#upload the kaggle.json file
from google.colab import files
files.upload()

#create a kaggle directory
!mkdir ~/.kaggle

#copy the kaggle.json to kaggle directory
!cp kaggle.json ~/.kaggle/

#permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!kaggle competitions download -c tweet-sentiment-extraction

  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 152MB/s]


In [3]:
!unzip tweet-sentiment-extraction.zip

Archive:  tweet-sentiment-extraction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [4]:
!pip install datatable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96.9 MB 114 kB/s 


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../content/train.csv')

In [3]:
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
df.shape

(27481, 4)

In [5]:
df.isnull().sum()

textID           0
text             1
selected_text    1
sentiment        0
dtype: int64

In [6]:
df = df.dropna()
df.isnull().sum()

textID           0
text             0
selected_text    0
sentiment        0
dtype: int64

In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.3 MB/s eta 0:14:53tcmalloc: large alloc 1147494400 bytes == 0x390de000 @  0x7f0be2a97615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |█████████████████               | 1055.7 MB 1.2 MB/s eta 0:12:23tcmalloc: large alloc 1434370048 bytes == 0x7d734000 @  0x7f0be2a97615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |█████

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [9]:
#Instantiate Model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [10]:
def sentiment_score(text):
    tokens = tokenizer.encode(text, return_tensors='pt')
    result = model(tokens)
    return torch.max(result.logits[0])

In [ ]:
df['bert_sentiment'] = df['text'].apply(sentiment_score)

In [ ]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.2 MB/s 


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
def vader_scores(feedbacktext, category):
    return vader.polarity_scores(feedbacktext).get(category)

vader = SentimentIntensityAnalyzer()

In [ ]:
df["Negative_Score"] = df.apply(lambda row : vader_scores(df["text"][row.name], "neg"),axis=1)
df["Neutral_Score"] = df.apply(lambda row : vader_scores(df["text"][row.name], "neu"),axis=1)
df["Positive_Score"] = df.apply(lambda row : vader_scores(df["text"][row.name], "pos"),axis=1)
df["Compound_Score"] = df.apply(lambda row : vader_scores(df["text"][row.name], "compound"),axis=1)

In [ ]:
from textblob import TextBlob

In [ ]:
#Create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

In [ ]:
#Create new columns ‘Subjectivity’
df['TextBlob_Subjectivity'] = df['text'].apply(getSubjectivity)

In [ ]:
#Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
#Create new columns ‘Polarity’
df ['TextBlob_Polarity'] = df['text'].apply(getPolarity)

In [ ]:
#Function for addressing polarity
def getAnalysis(score):
  if score < 0:
    return 'negative'
  elif score == 0:
    return 'neutral'
  else:
    return 'positive'

df ['TextBlob_Analysis'] = df['TextBlob_Polarity'].apply(getAnalysis )

In [ ]:
#Function for addressing polarity
def getAnalysis(score):
  if score <= -0.05:
    return 'negative'
  elif -0.05 < score < 0.05:
    return 'neutral'
  else:
    return 'positive'

df ['VADER_Analysis'] = df['Compound_Score'].apply(getAnalysis )

In [ ]:
# Install flair
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 5.2 MB/s 
     |████████████████████████████████| 1.6 MB 46.3 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 788 kB 41.6 MB/s 
     |████████████████████████████████| 1.2 MB 48.3 MB/s 
     |████████████████████████████████| 163 kB 52.9 MB/s 
     |████████████████████████████████| 46 kB 4.3 MB/s 
     |████████████████████████████████| 5.5 MB 43.5 MB/s 
     |████████████████████████████████| 981 kB 46.3 MB/s 
     |████████████████████████████████| 19.7 MB 1.2 MB/s 
     |████████████████████████████████| 200 kB 57.1 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 7.6 MB 40.4 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116701 sha256=92c88acbd3278d3ec27ca896d94285507ad1a645cc1f12f2c3276429bf84c9db
  Store

In [ ]:
# Import flair pre-trained sentiment model & Sentence to process input text
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')


2022-11-08 23:51:47,979 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmp_82_4uix


100%|██████████| 265512723/265512723 [00:12<00:00, 21072627.89B/s]

2022-11-08 23:52:00,975 copying /tmp/tmp_82_4uix to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2022-11-08 23:52:01,721 removing temp file /tmp/tmp_82_4uix
2022-11-08 23:52:01,769 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Define a function to get Flair sentiment prediction score
def score_flair(text):
  sentence = Sentence(text)
  classifier.predict(sentence)
  score = sentence.labels[0].score
  value = sentence.labels[0].value
  return score, value

In [ ]:
# Get sentiment score for each review
df['scores_flair'] = df['text'].apply(lambda s: score_flair(s)[0])

# Predict sentiment label for each review
df['pred_flair'] = df['text'].apply(lambda s: score_flair(s)[1])

In [ ]:
df['pred_flair_lc'] = df['pred_flair'].apply(lambda s: s.lower())

In [ ]:
# Import accuracy_score to check performance
from sklearn.metrics import accuracy_score
# Compare Actual and Predicted
print("TextBlob accuracy score: ", accuracy_score(df['sentiment'],df['TextBlob_Analysis']))
print("VADER accuracy score: ", accuracy_score(df['sentiment'],df['VADER_Analysis']))
print("Flair accuracy score: ", accuracy_score(df['sentiment'],df['pred_flair_lc']))

TextBlob accuracy score:  0.5861353711790394
VADER accuracy score:  0.6362809315866085
Flair accuracy score:  0.49370451237263463
